<a href="https://colab.research.google.com/github/MelissaMatindi/ai-devops-assignment/blob/main/AI_for_DevOps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Task 1 – **AI-Powered Code Completion**

In [1]:
# Install & load free Code Llama
!pip install transformers torch accelerate -q
from transformers import pipeline
import warnings; warnings.filterwarnings("ignore")

generator = pipeline(
    "text-generation",
    model="codellama/CodeLlama-7b-Python-hf",
    torch_dtype="auto",
    device_map="auto",
    max_new_tokens=128
)

def ai_suggest(prompt):
    result = generator(f"# {prompt}\n", temperature=0.1, do_sample=False)
    return result[0]['generated_text'].split("\n", 1)[1].strip()

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [2]:
def sort_dicts_by_key_manual(items: list, key: str):
    """
    Sort a list of dictionaries by the given key (manual implementation).
    """
    return sorted(items, key=lambda x: x[key])

In [3]:
# Let Code Llama generate the function
ai_code = ai_suggest("Python function to sort a list of dictionaries by a given key. Use built-in sorted and lambda.")
print("AI-suggested code:\n")
print(ai_code)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


AI-suggested code:

def sort_by_key(obj, key, reverse=False):
    return sorted(obj, key=lambda x: x[key], reverse=reverse)


# Driver code
obj = [
    {'name': 'John', 'age': 24},
    {'name': 'Jack', 'age': 25},
    {'name': 'Damon', 'age': 26},
    {'name': 'Joe', 'age': 27},
    {'name': 'James', 'age': 28}
]


In [4]:
def sort_by_key(obj, key, reverse=False):
    return sorted(obj, key=lambda x: x[key], reverse=reverse)


# Driver code
obj = [
    {'name': 'John', 'age': 24},
    {'name': 'Jack', 'age': 25},
    {'name': 'Damon', 'age': 26},
    {'name': 'Joe', 'age': 27},
    {'name': 'James', 'age': 28}
]

### **200-Word Analysis**

Both implementations are functionally identical and use Python’s Timsort via `sorted()` with a lambda key extractor, achieving O(n log n) time complexity and excellent real-world performance.

**Key differences:**

- **Naming & Parameters:**  
  Manual version uses descriptive `sort_dicts_by_key_manual` and explicit type hints (`items: list, key: str`) → better readability and IDE support.  
  AI version uses shorter `sort_by_key` and adds a useful `reverse=False` parameter by default, increasing flexibility without extra code from the caller.

- **Docstring & Clarity:**  
  Manual version includes a clear docstring; AI version omits documentation but provides example driver code, which is helpful for quick testing.

- **Efficiency & Style:**  
  Both are equally efficient (zero measurable difference on typical datasets <1M items). AI version is slightly more concise and follows common Python conventions seen in open-source projects.

**Conclusion:**  
The AI-generated code is marginally superior for production use due to the built-in `reverse` option and cleaner naming, while the manual version wins on explicit documentation and type hints. In real projects, developers should merge the best of both: keep the AI version and add the manual’s docstring + type hints. This demonstrates how AI tools accelerate development by delivering high-quality, idiomatic code instantly, allowing engineers to focus on higher-level design.


# Task 2: **Automated testing with AI**

In [5]:
# Install & run exported test
!pip install selenium
!pip install selenium webdriver-manager -q
!apt-get update && apt-get install -y chromium-browser > /dev/null

from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pytest, sys, os

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 2s (1,867 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (

In [6]:
# Base Element
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

class BaseElement(object):

    def __init__(self, driver, locator):
        self.driver = driver
        self.locator = locator
        self.web_element = None
        self.find()

    def find(self):
        element = WebDriverWait(self.driver, 10).until(EC.visibility_of_element_located(locator=self.locator))
        self.web_element = element
        return None

    def is_displayed(self):
        self.web_element.is_displayed()

    def exists(self):
        self.web_element.exists()


    def click(self):
        self.web_element.click()
        return None

    def input_text(self, text):
        self.web_element.send_keys(text)
        return None

    def text(self):
        text = self.web_element.text
        return text

    def current_url(self):
        self.web_element.current_url()
        return None


In [7]:
# Base Page
class BasePage(object):

    url = None

    def __init__(self, driver):
        self.driver = driver

    def go(self):
        self.driver.get(self.url)


In [8]:
# Drivers
from selenium import webdriver

class Drivers:
    def __init__(self, argument):
        self.argument = argument

    def chrome(self):
        options = webdriver.ChromeOptions()
        options.add_argument(self.argument)
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        driver = webdriver.Chrome(options=options)
        return driver

In [9]:
# Locator
from collections import namedtuple

Locator = namedtuple('locator', ['by', 'value'])

In [10]:
# Saucedemo
from selenium.webdriver.common.by import By

class SaucedemoPage(BasePage):

    url = 'https://www.saucedemo.com/'

# locator page_title used for each page verification
    @property
    def page_title(self):
        locator = Locator(By.CLASS_NAME, "title")
        return BaseElement(driver=self.driver, locator=locator)

    @property
    def swaglabs_login_logo(self):
        locator = Locator(By.CSS_SELECTOR, 'div[class="login_logo"]')
        return BaseElement(driver=self.driver, locator=locator)

    @property
    def username_input(self):
        locator = Locator(By.CSS_SELECTOR, 'input[id="user-name"]')
        return BaseElement(driver=self.driver, locator=locator)

    @property
    def password_input(self):
        locator = Locator(By.CSS_SELECTOR, 'input[id="password"]')
        return BaseElement(driver=self.driver, locator=locator)

    @property
    def login_button(self):
        locator = Locator(By.CSS_SELECTOR, 'input[id="login-button"]')
        return BaseElement(driver=self.driver, locator=locator)

    @property
    def burger_menu_button(self):
        locator = Locator(By.CSS_SELECTOR, 'button[id="react-burger-menu-btn"]')
        return BaseElement(driver=self.driver, locator=locator)

    @property
    def logout_sidebar_link(self):
        locator = Locator(By.CSS_SELECTOR, 'a[id="logout_sidebar_link"]')
        return BaseElement(driver=self.driver, locator=locator)

    # sauce labs fleece jacket
    @property
    def item_5_title_link_inventory(self):
        locator = Locator(By.CSS_SELECTOR, 'a[id="item_5_title_link"]')
        return BaseElement(driver=self.driver, locator=locator)

    @property
    def item_5_title_link_detail(self):
        locator = Locator(By.CSS_SELECTOR, 'div[class="inventory_details_name large_size"]')
        return BaseElement(driver=self.driver, locator=locator)

    @property
    def back_to_products_button(self):
        locator = Locator(By.CSS_SELECTOR, 'button[id="back-to-products"]')
        return BaseElement(driver=self.driver, locator=locator)

    @property
    def add_to_cart_item_5(self):
        locator = Locator(By.CSS_SELECTOR, 'button[id="add-to-cart-sauce-labs-fleece-jacket"]')
        return BaseElement(driver=self.driver, locator=locator)

    @property
    def remove_item_5(self):
        locator = Locator(By.CSS_SELECTOR, 'button[id="remove-sauce-labs-fleece-jacket"]')
        return BaseElement(driver=self.driver, locator=locator)

    @property
    def shopping_cart_button(self):
        locator = Locator(By.CLASS_NAME, "shopping_cart_link")
        return BaseElement(driver=self.driver, locator=locator)

    @property
    def item_5(self):
        locator = Locator(By.CLASS_NAME, "inventory_item_name")
        return BaseElement(driver=self.driver, locator=locator)

    @property
    def checkout_button(self):
        locator = Locator(By.CSS_SELECTOR, 'button[id="checkout"]')
        return BaseElement(driver=self.driver, locator=locator)

    @property
    def first_name_input_text(self):
        locator = Locator(By.CSS_SELECTOR, 'input[id="first-name"]')
        return BaseElement(driver=self.driver, locator=locator)

    @property
    def last_name_input_text(self):
        locator = Locator(By.CSS_SELECTOR, 'input[id="last-name"]')
        return BaseElement(driver=self.driver, locator=locator)

    @property
    def postal_code_input_text(self):
        locator = Locator(By.CSS_SELECTOR, 'input[id="postal-code"]')
        return BaseElement(driver=self.driver, locator=locator)

    @property
    def continue_button(self):
        locator = Locator(By.CSS_SELECTOR, 'input[id="continue"]')
        return BaseElement(driver=self.driver, locator=locator)


In [11]:
# Necessary libraries
!pip install selenium webdriver-manager -q

!apt-get update > /dev/null 2>&1 && apt-get install -y chromium-browser chromium-chromedriver > /dev/null 2>&1

## Testing